In [54]:
import glob
# This file should run basically insta fast. Bad coding for sure 
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "04" # Which month and days result you want to visualize in table
month = 11
repetitionValue = 1
repetitionName = "udrlBC3CVTest"
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.1
print(len(savedResults))
print(len(savedResults)/9)

90
10.0


In [55]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
topThreeAllSubjects = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    topThreeThisSubject = []
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()

            
            
            if len(topThreeThisSubject) < 2 or topThreeThisSubject[0][0] < res[1]:

                if len(topThreeThisSubject) == 2:
                    topThreeThisSubject.reverse()
                    topThreeThisSubject.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topThreeThisSubject.append([res[1], subNr, dataF, kernel, cVal])
                topThreeThisSubject.sort()
    if len(topThreeThisSubject)>0:
        topThreeAllSubjects.append(topThreeThisSubject)
# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 123)
(10, 123)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.245,0.25,0.26,0.205,0.2,0.2475,0.28,0.27,0.2375,0.2125,0.2025,0.2025,0.7025,0.705,0.6975,0.6625,0.6825,0.6525,0.2225,0.23,0.225,0.2825,0.3,0.275,0.245,0.245,0.255,0.715,0.7175,0.695,0.655,0.665,0.6625,0.225,0.265,0.2525,0.22,0.21,0.2475,0.71,0.7125,0.7,0.67,0.67,0.6575,0.295,0.285,0.24,0.7,0.71,0.6975,0.665,0.675,0.66,0.7025,0.705,0.7,0.6625,0.6825,0.6525,0.715,0.725,0.675,0.275,0.3025,0.2875,0.22,0.23,0.2275,0.71,0.72,0.6975,0.6525,0.66,0.66,0.2825,0.3,0.2825,0.7025,0.7125,0.695,0.6475,0.6675,0.665,0.715,0.7175,0.695,0.655,0.665,0.6625,0.7175,0.73,0.675,0.225,0.2425,0.2475,0.7075,0.71,0.695,0.6575,0.675,0.6625,0.71,0.7125,0.7,0.6675,0.665,0.66,0.72,0.7275,0.68,0.7,0.7075,0.6975,0.665,0.675,0.66,0.7175,0.7325,0.6825,0.715,0.725,0.675
2,0.258333,0.239583,0.214583,0.241667,0.235417,0.20625,0.29375,0.295833,0.25625,0.20625,0.241667,0.2125,0.725,0.747917,0.745833,0.76875,0.754167,0.7125,0.25,0.25,0.208333,0.25,0.21875,0.210417,0.258333,0.229167,0.214583,0.729167,0.739583,0.745833,0.760417,0.758333,0.71875,0.252083,0.25,0.247917,0.235417,0.241667,0.210417,0.725,0.74375,0.739583,0.772917,0.7375,0.7,0.295833,0.29375,0.252083,0.729167,0.739583,0.733333,0.770833,0.75625,0.716667,0.725,0.747917,0.745833,0.76875,0.747917,0.7125,0.747917,0.7625,0.729167,0.239583,0.222917,0.21875,0.247917,0.247917,0.208333,0.735417,0.7375,0.735417,0.76875,0.747917,0.71875,0.25,0.216667,0.214583,0.74375,0.733333,0.739583,0.764583,0.754167,0.722917,0.729167,0.739583,0.745833,0.760417,0.760417,0.71875,0.747917,0.760417,0.720833,0.252083,0.2375,0.24375,0.733333,0.733333,0.73125,0.764583,0.754167,0.708333,0.725,0.74375,0.739583,0.770833,0.7375,0.7,0.75,0.754167,0.73125,0.729167,0.739583,0.73125,0.770833,0.760417,0.716667,0.752083,0.766667,0.727083,0.747917,0.7625,0.729167
3,0.277778,0.266667,0.255556,0.244444,0.197222,0.225,0.291667,0.269444,0.291667,0.230556,0.225,0.222222,0.75,0.75,0.722222,0.716667,0.711111,0.713889,0.266667,0.255556,0.244444,0.319444,0.280556,0.263889,0.277778,0.263889,0.261111,0.75,0.75,0.719444,0.702778,0.713889,0.711111,0.197222,0.211111,0.2,0.227778,0.188889,0.225,0.747222,0.736111,0.716667,0.708333,0.705556,0.708333,0.280556,0.277778,0.283333,0.758333,0.752778,0.711111,0.719444,0.733333,0.713889,0.75,0.75,0.722222,0.719444,0.708333,0.713889,0.722222,0.738889,0.725,0.244444,0.225,0.2,0.266667,0.258333,0.241667,0.741667,0.738889,0.716667,0.702778,0.694444,0.716667,0.319444,0.280556,0.263889,0.752778,0.752778,0.711111,0.708333,0.741667,0.711111,0.75,0.75,0.719444,0.702778,0.713889,0.711111,0.719444,0.733333,0.716667,0.2,0.233333,0.188889,0.755556,0.744444,0.702778,0.711111,0.719444,0.711111,0.747222,0.736111,0.716667,0.705556,0.705556,0.711111,0.719444,0.725,0.725,0.758333,0.747222,0.711111,0.719444,0.730556,0.713889,0.708333,0.736111,0.716667,0.722222,0.738889,0.722222
4,0.241667,0.208333,0.210417,0.25,0.241667,0.1625,0.25625,0.19375,0.23125,0.214583,0.195833,0.216667,0.714583,0.675,0.729167,0.710417,0.7125,0.7,0.26875,0.2125,0.160417,0.2375,0.191667,0.

In [56]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.77 : subject 2, feature:welchDataBC--welchData, kernellinear, C:0.5
Accuracy 0.77 : subject 2, feature:welchDataBC-dataWCV--welchData, kernellinear, C:0.5
Accuracy 0.77 : subject 2, feature:welchDataBC-dataWCV--dataFFTCV, kernellinear, C:0.5
Accuracy 0.77 : subject 2, feature:welchDataBC--dataFFTCV, kernellinear, C:0.5
Accuracy 0.77 : subject 5, feature:welchDataBC-welchData--fftData, kernellinear, C:0.5
Accuracy 0.77 : subject 2, feature:welchDataBC-welchData--fftData, kernellinear, C:0.5
Accuracy 0.77 : subject 2, feature:-welchDataBC, kernellinear, C:0.5
Accuracy 0.77 : subject 2, feature:welchDataBC--dataWCV, kernellinear, C:0.5
Accuracy 0.77 : subject 2, feature:welchDataBC-fftDataBC--dataFFTCV, kernelsigmoid, C:2.5
Accuracy 0.76 : subject 2, feature:welchDataBC-dataFFTCV--fftData, kernellinear, C:0.5


In [57]:
print("Top ten highest accuracies")
#del(topThreeThisSubject[0])
#len(topThreeAllSubjects[0])
for subTop in topThreeAllSubjects:
    subTop.reverse()
topThreeAllSubjects.sort(reverse=True)
for subTop in topThreeAllSubjects:
    print("")
    #subTop.reverse()
    for top in subTop:
        print(
                f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
            )

Top ten highest accuracies

Accuracy 0.77 : subject 2, feature:welchDataBC--welchData, kernellinear, C:0.5
Accuracy 0.77 : subject 2, feature:welchDataBC-dataWCV--welchData, kernellinear, C:0.5

Accuracy 0.77 : subject 5, feature:welchDataBC-welchData--fftData, kernellinear, C:0.5
Accuracy 0.76 : subject 5, feature:fftDataBC-dataFFTCV--fftData, kernellinear, C:0.5

Accuracy 0.76 : subject 3, feature:fftDataBC-dataWCV--dataFFTCV, kernellinear, C:0.5
Accuracy 0.76 : subject 3, feature:fftDataBC--dataFFTCV, kernellinear, C:0.5

Accuracy 0.75 : subject 7, feature:welchDataBC-dataWCV--fftData, kernellinear, C:0.5
Accuracy 0.75 : subject 7, feature:welchDataBC--fftData, kernellinear, C:0.5

Accuracy 0.74 : subject 4, feature:welchDataBC-welchData--fftData, kernelsigmoid, C:2.5
Accuracy 0.74 : subject 4, feature:welchDataBC--fftData, kernelsigmoid, C:2.5

Accuracy 0.73 : subject 1, feature:welchDataBC-fftDataBC--dataFFTCV, kernelsigmoid, C:2.5
Accuracy 0.73 : subject 1, feature:welchDataBC-ff

In [58]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)

print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(headers[maxInd])
print(nrOfSubjects[maxInd])
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")


[0.         0.53853659 0.57012195 0.56246612 0.53995596 0.5638042
 0.5272912  0.55430217 0.52542683 0.53721206]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122
0,0.249555,0.235636,0.2314,0.246399,0.239485,0.231494,0.240349,0.248486,0.242315,0.216691,0.231334,0.221657,0.715711,0.701284,0.711072,0.699558,0.706832,0.696177,0.248649,0.235983,0.225953,0.24868,0.241021,0.232406,0.249555,0.235832,0.229227,0.714735,0.702273,0.710485,0.702813,0.709047,0.69868,0.235659,0.242678,0.233329,0.242459,0.243808,0.234562,0.714426,0.702741,0.709003,0.70138,0.703791,0.692323,0.24494,0.243099,0.239773,0.712572,0.701474,0.705818,0.699732,0.703439,0.694102,0.715711,0.701515,0.71135,0.699332,0.707959,0.695504,0.72361,0.718837,0.704642,0.247472,0.234411,0.227062,0.24814,0.236755,0.226852,0.713931,0.705318,0.707378,0.704025,0.70469,0.695863,0.24868,0.237928,0.23289,0.71602,0.70131,0.70695,0.704773,0.707719,0.698112,0.714735,0.702967,0.710485,0.702813,0.709531,0.698427,0.723848,0.719348,0.704221,0.237567,0.242214,0.229212,0.716257,0.699875,0.704088,0.700599,0.706315,0.692918,0.714426,0.702973,0.709024,0.699662,0.70377,0.693836,0.724968,0.715678,0.703493,0.712572,0.700579,0.705355,0.699732,0.704561,0.694586,0.723018,0.718667,0.701982,0.72361,0.719068,0.704334


Max average accuracy 0.7249677328843995
108
9.0
welchDataBC-fftDataBC--welchData
linear
C: 0.5


In [59]:
x = 0
for names in nameDict.keys():
    print(names)
    x +=1
    print(x)

-fftData
1
-welchData
2
-dataFFTCV
3
-dataWCV
4
-fftDataBC
5
-welchDataBC
6
welchData--fftData
7
dataFFTCV--fftData
8
dataWCV--fftData
9
fftDataBC--fftData
10
welchDataBC--fftData
11
dataFFTCV--welchData
12
dataWCV--welchData
13
fftDataBC--welchData
14
welchDataBC--welchData
15
dataWCV--dataFFTCV
16
fftDataBC--dataFFTCV
17
welchDataBC--dataFFTCV
18
fftDataBC--dataWCV
19
welchDataBC--dataWCV
20
welchDataBC--fftDataBC
21
dataFFTCV-welchData--fftData
22
dataWCV-welchData--fftData
23
fftDataBC-welchData--fftData
24
welchDataBC-welchData--fftData
25
dataWCV-dataFFTCV--fftData
26
fftDataBC-dataFFTCV--fftData
27
welchDataBC-dataFFTCV--fftData
28
fftDataBC-dataWCV--fftData
29
welchDataBC-dataWCV--fftData
30
welchDataBC-fftDataBC--fftData
31
dataWCV-dataFFTCV--welchData
32
fftDataBC-dataFFTCV--welchData
33
welchDataBC-dataFFTCV--welchData
34
fftDataBC-dataWCV--welchData
35
welchDataBC-dataWCV--welchData
36
welchDataBC-fftDataBC--welchData
37
fftDataBC-dataWCV--dataFFTCV
38
welchDataBC-dataWCV--

In [60]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122
0,2.245997,2.12072,2.082601,2.217588,2.155366,2.083447,2.163144,2.236376,2.180833,1.950215,2.082008,1.994912,6.441402,6.311553,6.399646,6.296023,6.36149,6.265593,2.237841,2.123851,2.033573,2.238119,2.169192,2.091654,2.245997,2.122487,2.063043,6.432614,6.320455,6.394369,6.325316,6.381427,6.288119,2.120934,2.184104,2.099962,2.182134,2.194268,2.111061,6.429836,6.324672,6.381023,6.312424,6.334116,6.230909,2.204457,2.187891,2.157955,6.413144,6.31327,6.352361,6.297588,6.330947,6.246919,6.441402,6.313636,6.402146,6.29399,6.371629,6.259533,6.512487,6.469533,6.34178,2.227247,2.109697,2.043561,2.233258,2.130795,2.041667,6.425379,6.347866,6.366402,6.336225,6.34221,6.262765,2.238119,2.141351,2.09601,6.444179,6.311793,6.362551,6.342955,6.36947,6.283005,6.432614,6.326705,6.394369,6.325316,6.385783,6.285846,6.514634,6.474129,6.337992,2.138106,2.179924,2.062904,6.446313,6.298876,6.336793,6.305391,6.356831,6.236263,6.429836,6.326755,6.381212,6.296957,6.333927,6.24452,6.52471,6.441098,6.331439,6.413144,6.305215,6.348194,6.297588,6.341048,6.251275,6.507159,6.468005,6.317841,6.512487,6.471616,6.339003


108
9.0
Max total significant accuracy then averaged 0.7249677328843995 
welchDataBC-fftDataBC--welchData
linear
C: 0.5 


In [61]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
